In [36]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [37]:
# method = METHOD_MITHRAL
method = METHOD_PQ
# method = METHOD_EXACT

In [38]:
feedback_bits = 256
ncodebooks=32
ncentroids=2048
train_sam_num = 3000 # 训练集样本数

In [39]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [40]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  PQ
running kmeans in subspace 1/32... mse / {var(X_subs), var(X)}: 0.123, 0.00154
running kmeans in subspace 2/32... mse / {var(X_subs), var(X)}: 0.121, 0.00934
running kmeans in subspace 3/32... mse / {var(X_subs), var(X)}: 0.131, 0.00267
running kmeans in subspace 4/32... mse / {var(X_subs), var(X)}: 0.149, 0.0153
running kmeans in subspace 5/32... mse / {var(X_subs), var(X)}: 0.161, 0.00255
running kmeans in subspace 6/32... mse / {var(X_subs), var(X)}: 0.199, 0.017
running kmeans in subspace 7/32... mse / {var(X_subs), var(X)}: 0.157, 0.00288
running kmeans in subspace 8/32... mse / {var(X_subs), var(X)}: 0.192, 0.0184
running kmeans in subspace 9/32... mse / {var(X_subs), var(X)}: 0.135, 0.00151
running kmeans in subspace 10/32... mse / {var(X_subs), var(X)}: 0.123, 0.00926
running kmeans in subspace 11/32... mse / {var(X_subs), var(X)}: 0.134, 0.00269
running kmeans in subspace 12/32... mse / {var(X_subs), var(X)}: 0.147, 0.015
running kmeans in subspace 13/32...

In [41]:
x_test = np.load(dir_train+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [42]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.08786726 -0.17161494 -0.4597496  ... -0.24521011  0.02531986
  -0.14386548]
 [-0.25566763 -0.13506436 -0.3135473  ... -0.38642824  0.00538318
  -0.10897629]
 [ 0.16964817 -0.06860878 -0.11251915 ... -0.40138075  0.12001908
   0.11697274]
 ...
 [ 0.2145057   0.05267269 -0.50128436 ...  0.00399838 -0.11755967
   0.06380826]
 [ 0.22447404  0.12577385 -0.3318226  ...  0.15352347  0.0668546
  -0.12060602]
 [-0.39024022  0.21715029 -0.43981293 ... -0.46783635 -0.0145535
   0.16681443]]
(32000, 256)
